In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_amsterdam = pd.read_csv('data_amsterdam.csv')

In [3]:
data_amsterdam.columns

Index(['Unnamed: 0', 'realSum', 'person_capacity', 'multi', 'biz',
       'cleanliness_rating', 'guest_satisfaction_overall', 'bedrooms', 'dist',
       'metro_dist', 'attr_index_norm', 'rest_index_norm', 'lng', 'lat',
       'room_shared_True', 'room_private_True', 'host_is_superhost_True',
       'weekend_yes'],
      dtype='object')

In [4]:
data_amsterdam = data_amsterdam.drop(['Unnamed: 0'], axis=1)

In [5]:
from scipy import stats

# Assuming df is the name of your dataframe
z_scores = np.abs(stats.zscore(data_amsterdam))

# Remove rows where any of the columns have a z-score greater than 3
data_amsterdam_cleaned = data_amsterdam[(z_scores < 3).all(axis=1)]

In [6]:
data_amsterdam_cleaned.corr()

,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,lng,lat,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
realSum,1.000000,0.582634,-0.166164,-0.019447,-0.015354,0.124446,0.504269,-0.321887,-0.202476,0.349669,0.335122,0.050536,0.027782,NaN,-0.519748,-0.052223,0.099896
person_capacity,0.582634,1.000000,-0.136097,0.023386,-0.109162,0.002492,0.648573,-0.034859,-0.027224,0.054775,0.029041,0.008621,0.025935,NaN,-0.324242,-0.065245,-0.007936
multi,-0.166164,-0.136097,1.000000,-0.201104,0.030018,-0.058973,-0.093250,0.029811,-0.035614,0.012917,0.017325,-0.014803,-0.037731,NaN,0.281475,0.079652,-0.063681
biz,-0.019447,0.023386,-0.201104,1.000000,-0.036604,-0.171331,-0.097021,-0.112173,-0.085945,0.107361,0.132265,0.044473,0.062876,NaN,0.085315,-0.113858,-0.031670
cleanliness_rating,-0.015354,-0.109162,0.030018,-0.036604,1.000000,0.559087,-0.018202,-0.028639,-0.015177,0.004992,0.012495,-0.025834,0.000225,NaN,0.027985,0.298627,0.002591
guest_satisfaction_overall,0.124446,0.002492,-0.058973,-0.171331,0.559087,1.000000,0.109102,-0.030083,0.007462,-0.040911,-0.031476,-0.029906,0.026570,NaN,-0.177432,0.293309,0.037701
bedrooms,0.504269,0.648573,-0.093250,-0.097021,-0.018202,0.109102,1.000000,0.073893,0.014752,-0.074199,-0.083189,0.004473,-0.050510,NaN,-0.353769,-0.044329,0.015658
dist,-0.321887,-0.034859,0.029811,-0.112173,-0.028639,-0.030083,0.073893,1.000000,0.422075,-0.794082,-0.787015,-0.097483,-0.366751,NaN,0.089108,0.017159,-0.007378
metro_dist,-0.202476,-0.027224,-0.035614,-0.085945,-0.015177,0.007462,0.014752,0.422075,1.000000,-0.486375,-0.474267,-0.058042,0.306861,NaN,0.039108,-0.004491,0.004549
attr_index_norm,0.349669,0.054775,0.012917,0.107361,0.004992,-0.040911,-0.074199,-0.794082,-0.486375,1.000000,0.922032,0.059840,0.116621,NaN,0.014121,0.006061,-0.023384


In [37]:
data_amsterdam_cleaned_reg = data_amsterdam_cleaned.drop(['cleanliness_rating', 'host_is_superhost_True', 'lat', 'lng'], axis=1)

In [38]:
x = data_amsterdam_cleaned_reg.drop('realSum', axis=1)
y = data_amsterdam_cleaned_reg.realSum

In [39]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,multi,biz,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,weekend_yes
count,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.0,1371.000000,1371.000000
mean,2.705325,0.278629,0.097009,95.099198,1.252371,2.595427,1.038381,14.103346,26.282005,0.0,0.451495,0.464624
std,0.938609,0.448488,0.296079,4.698696,0.669829,1.654042,0.704264,8.546494,16.583811,0.0,0.497823,0.498929
min,2.000000,0.000000,0.000000,76.000000,0.000000,0.110000,0.040000,2.719044,4.463215,0.0,0.000000,0.000000
25%,2.000000,0.000000,0.000000,93.000000,1.000000,1.360000,0.480000,7.377210,13.644861,0.0,0.000000,0.000000
50%,2.000000,0.000000,0.000000,96.000000,1.000000,2.290000,0.870000,11.226431,21.295415,0.0,0.000000,0.000000
75%,4.000000,1.000000,0.000000,99.000000,2.000000,3.475000,1.500000,19.769228,34.555717,0.0,1.000000,1.000000
max,5.000000,1.000000,1.000000,100.000000,3.000000,8.890000,3.510000,44.685530,79.048991,0.0,1.000000,1.000000


In [40]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     191.9
Date:                Thu, 09 Mar 2023   Prob (F-statistic):          3.04e-267
Time:                        22:29:16   Log-Likelihood:                -9012.2
No. Observations:                1371   AIC:                         1.805e+04
Df Residuals:                    1359   BIC:                         1.811e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [36]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [12]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.607
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     263.3
Date:                Thu, 09 Mar 2023   Prob (F-statistic):          4.34e-270
Time:                        15:12:40   Log-Likelihood:                -9014.0
No. Observations:                1371   AIC:                         1.805e+04
Df Residuals:                    1362   BIC:                         1.809e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        542.1346      4.699    115.360      0.0

In [13]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

-266.697539690345

In [14]:
model.score(x_train, y_train)

0.6072930910670564

In [15]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [16]:
mse(y_test,y_pred)

33741.78540823786

In [17]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

183.68937206119972

In [18]:
##prediction on the train set
mse(y_train,y_pred_train)

30079.98789222721

In [19]:
mae(y_test,y_pred)

131.54078061791174

In [20]:
R2=r2_score(y_test,y_pred)
R2

0.5748493010738933

In [21]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.5672742329415796

In [22]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.6072922699486087, Test -> 0.5748922804013428


In [23]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.5089964624463581, Test -> 0.4781483667799168


In [24]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 1371 rows and 8 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.63
The coefficient of determination for the train data is R2=1.00


In [25]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.33046035 0.46604651 0.53377354 0.47547692 0.41331975]
Score stats: 0.44 accuracy with a standard deviation of 0.07


In [26]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': 0.407051213105055, 'Linear Regression': 0.5934955500205241, 'KNN': 0.5100305284570851}


In [27]:
data_amsterdam_cleaned.to_excel('data_amsterdam.xlsx')